# Ausrüster AG Use Case

## Python Notebook: FOM - Area of Application - Business Analytics

Author: Dr. Stephan Hausberg, Winter semester 2024

Data Retrieval of external geographical data: Open Street Map



In [ ]:
import pandas as pd
import numpy as np
import overpy
import requests

boundbox = (50.671651, 6.960198, 50.855700, 7.283077)

### Data retrieval
def get_data(bbox: tuple, amenity_spec : str ) -> object:
    ###
    api = overpy.Overpass()
    a, b, c, d = bbox[0], bbox[1], bbox[2], bbox[3]
    long_str = (
        '[out:json][timeout:200];(node["amenity" = "' + amenity_spec + '"](%s,%s,%s,%s););out body;>;out skel qt;'
        % (a, b, c, d)
    )
    result = api.query(long_str)
    return result
res_data = get_data(boundbox, "doctors")

In [ ]:
res_data
#res_data.nodes
#res_data.nodes[4]
#res_data.nodes[4].id
#res_data.nodes[4].tags

In [ ]:
def get_highway(bbox):
    ###
    api = overpy.Overpass()
    a, b, c, d = bbox[0], bbox[1], bbox[2], bbox[3]
    long_str = (
        '[out:json][timeout:200];(way["highway"](%s,%s,%s,%s););out body;>;out skel qt;'
        % (a, b, c, d)
    )
    result = api.query(long_str)
    return result

boundbox_small = (50.67, 6.95, 50.68, 6.99)
res_highway = get_highway(boundbox_small)

In [ ]:
res_highway.nodes
#res_highway.ways

How can we display all this data? The solution is called Folium!
First, we need to transform this into a different coordinate and dataframe setting.

In [ ]:
### Transformation to dataframe
def transform_data_to_df(long_dict) -> pd.DataFrame:
    cols = ['id', 'lat', 'lon', 'name', 'label']
    df = pd.DataFrame(columns = cols)
    for i in range(0,len(long_dict.nodes)):
        data = long_dict.nodes[i]
        try:
            data.tags['name']
            name = data.tags['name']
        except:
            name = ""
        try:
            data.tags['amenity']
            label = data.tags['amenity']
        except:
            name = ""
        locator = [int(data.id), float(data.lat), float(data.lon), name, label]
        df_1 = pd.DataFrame.from_records([locator])
        df_1.columns = cols
        df = pd.concat([df, df_1], ignore_index = True)
    return df


### Execution
cols = ['id', 'lat', 'lon', 'name', 'label']
df_out = pd.DataFrame(columns = cols)

for j in ['doctors', 'hospital', 'clinic']:
    res_data = get_data(boundbox, j)
    df_out = pd.concat([df_out, transform_data_to_df(res_data)])

df_out = df_out.sort_values(by = 'id').reset_index(drop = True)

In [ ]:
df_out.head()

In [ ]:
dict_highway = dict()
dict_nodes = dict()
for j in res_highway.ways:
    liste = []
    for i in j.nodes:
        liste.append([float(i.lat), float(i.lon)])
        dict_nodes[i.id] = [float(i.lat), float(i.lon)]
    dict_highway[j.id] = liste

In [ ]:
dict_highway

Using Folium library to visualize data we have extracted from OSM.

In [ ]:
import folium as fl

m = fl.Map(
    location=[df_out.lat.median(), df_out.lon.median()],
    zoom_start=11,
    tiles="cartodbpositron",
)

#'openstreetmap'
#'mapquestopen'
#'Mapbox Control Room'
#'stamenwatercolor'


layer_raw = fl.FeatureGroup(name="Doctors",
                            show=False)

for i in range(0, df_out.shape[0]):
    fl.Circle(
        location=[df_out.iloc[i].lat, df_out.iloc[i].lon],
        radius=10,
        color="darkblue",
        opacity=0.8,
    ).add_to(layer_raw)
m.add_child(layer_raw)

fg = fl.FeatureGroup("Straßen", show=False)
for key in dict_highway:
    fl.PolyLine(dict_highway[key], opacity=0.50, weight=2, color="grey").add_to(fg)
fg.add_to(m)

fl.LayerControl().add_to(m)

m